In [11]:
%cd /content
!git clone https://github.com/GabrielSainz/PML_FINAL_ASSIGNMENT.git PML_FINAL_ASSIGNMENT
%cd PML_FINAL_ASSIGNMENT

/content
Cloning into 'PML_FINAL_ASSIGNMENT'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 0), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 648.61 KiB | 22.37 MiB/s, done.
/content/PML_FINAL_ASSIGNMENT


In [15]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

from exercise_1.utils.VAE import VAEConfig, train_vae

# -------------------------
# Device
# -------------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda:0


In [ ]:
root_path="/content/drive/MyDrive/University_of_Copenhagen/block6/PML/final_assignment/exercise_1" 

# Data

In [16]:
batch_size = 128
val_split = 0.1
num_workers = 2

transform = transforms.ToTensor()  # required for BCE loss in vae.py

dataset_full = datasets.MNIST(f"{root_path}/mnist_data", download=True, train=True, transform=transform)
n_val = int(len(dataset_full) * val_split)
n_train = len(dataset_full) - n_val
dataset_train, dataset_val = random_split(dataset_full, [n_train, n_val])

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.53MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 17.6MB/s]


# Beta-VAE 

In [ ]:
# -------------------------
# Experiment grid
# -------------------------
latent_dims = [2, 16]
betas = [0.01, 0.1, 1.0]

# Training knobs (shared)
epochs = 15
lr = 2e-3
seed = 42

run_root = f"{root_path}/runs_vae"  # parent folder


In [ ]:
# optional: keep a summary list
runs_summary = []

for zdim in latent_dims:
    for beta in betas:
        run_name = f"mnist_betaVAE_z{zdim}_beta{beta:g}_e{epochs}_lr{lr:g}_seed{seed}"
        print("\n" + "="*90)
        print("RUN:", run_name)
        print("="*90)

        cfg = VAEConfig(
            latent_dim=zdim,
            beta=float(beta),
            epochs=epochs,
            lr=lr,
            seed=seed,
            run_root=run_root,
            run_name=run_name,        # ensures separate folder per config
            save_best_only=True,

            # plot params
            max_images_plot=16,
            prior_n_plot=25,
            latent_grid_size=20,
            latent_grid_lim=3.0,
            latent_scatter_max_points=5000,
        )

        model, history, run_dir = train_vae(cfg, dataloader_train, dataloader_val, device)

        runs_summary.append({
            "run_name": run_name,
            "run_dir": run_dir,
            "latent_dim": zdim,
            "beta": beta,
            "final_val_loss": history["val_loss"][-1],
            "final_val_recon": history["val_recon"][-1],
            "final_val_kl": history["val_kl"][-1],
        })

print("\nAll runs finished.")


# DDPM Training